### Chord diagram using the Chord library (https://python-graph-gallery.com/chord-diagram-python-chord/)

TLDR: requires license but we can use d3blocks

In [13]:
from CONWAY_table_data import evo_table
import numpy as np
import pandas as pd


In [14]:
evo_mapping_dict = {row[1]: row[2].split() for row in evo_table}
column_names = ['atomic_num', 'element',
                'evolution', 'string', 'string_evolution']
df = pd.DataFrame(evo_table, columns=column_names)


In [15]:
df.head()

,atomic_num,element,evolution,string,string_evolution
0,1,H,H,22,22
1,2,He,Hf Pa H Ca Li,13112221133211322112211213322112,11132132212312211322212221121123222112
2,3,Li,He,312211322212221121123222112,13112221133211322112211213322112
3,4,Be,Ge Ca Li,111312211312113221133211322112211213322112,3113112221131112211322212312211322212221121123...
4,5,B,Be,1321132122211322212221121123222112,111312211312113221133211322112211213322112


In [16]:
evo_mapping_dict['He']

['Hf', 'Pa', 'H', 'Ca', 'Li']

In [17]:
def get_encoding(e):
    #print(e)
    one_hot = np.zeros((92))
    elements = evo_mapping_dict[e]
    #print(elements)
    indices = (df.loc[df['element'].isin(elements), 'atomic_num']-1).tolist()
    one_hot[indices] = 1
    return one_hot


In [18]:
def get_atom_label(i):
    return df.loc[df['atomic_num'] == i+1, 'element'].tolist()[0]


In [19]:
onehot = []
for e in df['element']:
    onehot.append(get_encoding(e))
onehot = np.array(onehot)
print(onehot.shape)


(92, 92)


In [20]:
# Create a DataFrame with the connection data
onehot_df = pd.DataFrame(onehot, columns=range(1, 93), index=range(1, 93))

labels = list(df['element'])
print(labels)

onehot_df.head()


['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'Cl', 'Ar', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As', 'Se', 'Br', 'Kr', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 'Sb', 'Te', 'I', 'Xe', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Po', 'At', 'Rn', 'Fr', 'Ra', 'Ac', 'Th', 'Pa', 'U']


,1,2,3,4,5,6,7,8,9,10,...,83,84,85,86,87,88,89,90,91,92
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
# Compute the number of connections for each element
num_connections = onehot_df.sum(axis=1)

# Create the source data for the chord diagram
data = onehot_df.stack().reset_index()
data.columns = ['source', 'target', 'value']
color = [str(num_connections[i]) for i in data['source']]
data['color'] = color

# Create a dictionary to map the atomic numbers to element names
atomic_number_to_element = {row[0]: row[1] for row in evo_table}

# Map the atomic numbers to element names in the data DataFrame
data['source_element'] = data['source'].map(atomic_number_to_element)
data['target_element'] = data['target'].map(atomic_number_to_element)

data = data[data['value'] != 0]
data = data.sort_values(by=['color'], ascending=False) # sort by the num-connections (outgoing), decreasing order
print(data.head())

      source  target  value color source_element target_element
1866      21      27    1.0   5.0             Sc             Co
1840      21       1    1.0   5.0             Sc              H
1906      21      67    1.0   5.0             Sc             Ho
1930      21      91    1.0   5.0             Sc             Pa
92         2       1    1.0   5.0             He              H


In [22]:
onehot_df.to_csv('./onehot_df.csv')

onehot_df.to_json('./onehot_df.json')


In [23]:
from chord import Chord

Chord(onehot_df.to_numpy().tolist(), names=labels).to_html('./chord_py.html')

Exception: {'reason': 'Authentication failed. Please check your license details or get one from https://plotapi.com.'}

requires a license :(

But we can use the d3blocks library (https://d3blocks.github.io/d3blocks/pages/html/Chord.html)

In [24]:
plot_data = data[['source_element', 'target_element', 'color']]

plot_data = plot_data.reset_index(
)[['source_element', 'target_element', 'color']]

plot_data.rename(columns={'source_element': 'source', 'target_element': 'target', 'color': 'weight'}, inplace=True)

plot_data.head()

,source,target,weight
0,Sc,Co,5.0
1,Sc,H,5.0
2,Sc,Ho,5.0
3,Sc,Pa,5.0
4,He,H,5.0


In [38]:
# Chord plot weighted by the number of connections
from d3blocks import D3Blocks

d3 = D3Blocks(chart='Chord', frame=False)
d3.set_node_properties(plot_data, opacity=0.9, cmap='tab20')
d3.set_edge_properties(plot_data, color='source', opacity=0.6)

d3.show()

# d3.chord(df=plot_data, filepath='./chord_py_d3_weighted.html')

[d3blocks] >INFO> Cleaning edge_properties and config parameters..
[d3blocks] >INFO> Initializing [Chord]
[d3blocks] >INFO> filepath is set to [C:\Users\Aditiya\AppData\Local\Temp\d3blocks\chord.html]
[d3blocks] >INFO> Node properties are set.
[d3blocks] >INFO> Convert to Dictionary.
[d3blocks] >INFO> Edge properties are set.
[d3blocks] >INFO> File already exists and will be overwritten: [C:\Users\Aditiya\AppData\Local\Temp\d3blocks\chord.html]
[d3blocks] >INFO> Open browser: C:\Users\Aditiya\AppData\Local\Temp\d3blocks\chord.html


In [30]:
# Unweighted chord plot
plot_data['weight'] = 1

d3.chord(df=plot_data, filepath='./chord_py_d3_unweighted.html')

[d3blocks] >INFO> Cleaning edge_properties and config parameters..
[d3blocks] >INFO> Initializing [Chord]
[d3blocks] >INFO> filepath is set to [d:\Master's Stuff Aditya\SWP Algorithms\swp_graph\conway graph\chord_py_d3_unweighted.html]
[d3blocks] >INFO> Node properties are set.
[d3blocks] >INFO> Set edge-opacity based on the [source] node-opacity.
[d3blocks] >INFO> Set edge-colors based on the [source] node-color.
[d3blocks] >INFO> Convert to Dictionary.
[d3blocks] >INFO> Edge properties are set.
[d3blocks] >INFO> File already exists and will be overwritten: [d:\Master's Stuff Aditya\SWP Algorithms\swp_graph\conway graph\chord_py_d3_unweighted.html]
[d3blocks] >INFO> Open browser: d:\Master's Stuff Aditya\SWP Algorithms\swp_graph\conway graph\chord_py_d3_unweighted.html
